In [1]:
import pandas as pd

In [7]:
rookie_df_22 = pd.read_csv("2022rookiestats.csv")
print(rookie_df_22.head())
print(rookie_df_22.columns.values)

   Unnamed: 0  Pick   Tm            Player Pos  Age      To  AP1  PB  St  ...  \
0           9     8  ATL      Drake London  WR   21  2024.0    0   0   3  ...   
1          11    10  NYJ    Garrett Wilson  WR   22  2024.0    0   0   3  ...   
2          12    11  NOR       Chris Olave  WR   22  2024.0    0   0   3  ...   
3          13    12  DET  Jameson Williams  WR   21  2024.0    0   0   1  ...   
4          17    16  WAS      Jahan Dotson  WR   22  2024.0    0   0   2  ...   

   (263, 'Coll_rush_td')  (263, 'Coll_rush_yds_per_g')  (263, 'Coll_rec')  \
0                     19                          24.5                  0   
1                     19                          24.5                  0   
2                     19                          24.5                  0   
3                     19                          24.5                  0   
4                     19                          24.5                  0   

   (263, 'Coll_rec_yds')  (263, 'Coll_rec_td')  (2

# Rookie QB Model:
- Games Played
- Completion Percentage
- pass attempts per game
- passing yds per game
- passing tds per game
- interceptions per game
- rush yds per game
- rush tds per game

In [ ]:
def compute_features(df):
    